# Intégration CoreML de RusTorch - Liaisons Python

Ce notebook démontre comment utiliser la fonctionnalité CoreML de RusTorch via les liaisons Python.

## Configuration et Importations

In [ ]:
# Importer les liaisons Python de RusTorch
try:
    import rustorch
    print(f"✅ Version de RusTorch : {rustorch.__version__}")
    print(f"📝 Description : {rustorch.__description__}")
    print(f"👥 Auteur : {rustorch.__author__}")
except ImportError as e:
    print(f"❌ Échec de l'importation de RusTorch : {e}")
    print("Veuillez construire avec maturin develop")
    exit(1)

import numpy as np
import platform

print(f"🖥️ Plateforme : {platform.system()} {platform.release()}")
print(f"🐍 Version Python : {platform.python_version()}")

## Vérifier la Disponibilité de CoreML

In [ ]:
# Vérifier la fonctionnalité CoreML
try:
    # Vérifier si CoreML est disponible
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML disponible : {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML est disponible !")
        
        # Obtenir les informations du périphérique
        device_info = rustorch.get_coreml_device_info()
        print("📱 Informations du périphérique CoreML :")
        print(device_info)
    else:
        print("⚠️ CoreML n'est pas disponible")
        if platform.system() != "Darwin":
            print("CoreML n'est disponible que sur macOS")
        else:
            print("Les fonctionnalités CoreML peuvent ne pas être activées")
            
except AttributeError:
    print("❌ Fonctions CoreML non trouvées")
    print("Peut ne pas être construit avec les fonctionnalités CoreML")
    coreml_available = False
except Exception as e:
    print(f"❌ Erreur lors de la vérification de CoreML : {e}")
    coreml_available = False

## Création de Périphérique CoreML et Opérations

In [ ]:
if coreml_available:
    try:
        # Créer un périphérique CoreML
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ Périphérique CoreML créé : {device}")
        
        # Obtenir les informations du périphérique
        print(f"🆔 ID du périphérique : {device.device_id()}")
        print(f"✅ Disponible : {device.is_available()}")
        print(f"💾 Limite de mémoire : {device.memory_limit()} octets")
        print(f"🧮 Limite des unités de calcul : {device.compute_units_limit()}")
        print(f"📚 Taille du cache du modèle : {device.model_cache_size()}")
        
        # Nettoyage du cache
        device.cleanup_cache()
        print("🧹 Cache nettoyé")
        
    except Exception as e:
        print(f"❌ Erreur d'opération du périphérique CoreML : {e}")
else:
    print("⚠️ Saut des opérations du périphérique car CoreML n'est pas disponible")

## Configuration du Backend CoreML

In [ ]:
if coreml_available:
    try:
        # Créer la configuration du backend CoreML
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Configuration du backend : {config}")
        
        # Vérifier et modifier les valeurs de configuration
        print(f"📊 Activer la mise en cache : {config.enable_caching}")
        print(f"🗂️ Taille maximale du cache : {config.max_cache_size}")
        print(f"📈 Activer le profilage : {config.enable_profiling}")
        print(f"🔄 Basculement automatique : {config.auto_fallback}")
        
        # Modifier la configuration
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Configuration mise à jour : {config}")
        
        # Créer le backend CoreML
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 Backend CoreML : {backend}")
        print(f"✅ Backend disponible : {backend.is_available()}")
        
        # Obtenir les statistiques du backend
        stats = backend.get_stats()
        print(f"📊 Statistiques du backend : {stats}")
        print(f"   Opérations totales : {stats.total_operations}")
        print(f"   Succès du cache : {stats.cache_hits}")
        print(f"   Échecs du cache : {stats.cache_misses}")
        print(f"   Opérations de basculement : {stats.fallback_operations}")
        print(f"   Taux de succès du cache : {stats.cache_hit_rate():.2%}")
        print(f"   Taux de basculement : {stats.fallback_rate():.2%}")
        print(f"   Temps d'exécution moyen : {stats.average_execution_time_ms:.2f}ms")
        
        # Nettoyage du cache
        backend.cleanup_cache()
        print("\n🧹 Cache du backend nettoyé")
        
    except Exception as e:
        print(f"❌ Erreur d'opération du backend CoreML : {e}")
else:
    print("⚠️ Saut des opérations du backend car CoreML n'est pas disponible")

## Opérations de Tenseurs de Base (CPU)

Pour comparer avec CoreML, effectuons d'abord des opérations de base sur CPU.

In [ ]:
try:
    # Création et opérations de tenseurs de base
    print("🧮 Opérations de tenseurs de base (CPU)")
    
    # Créer des tenseurs à partir de tableaux NumPy (interface simplifiée)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Forme de la matrice A : {data_a.shape}")
    print(f"📐 Forme de la matrice B : {data_b.shape}")
    
    # Multiplication de matrices avec NumPy (pour comparaison)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ Forme du résultat matmul NumPy : {numpy_result.shape}")
    print(f"📊 Résultat (premiers éléments) : {numpy_result.flatten()[:4]}")
    
    print("\n🚀 Opérations CPU terminées")
    
except Exception as e:
    print(f"❌ Erreur d'opération de tenseur : {e}")

## Simulation de Comparaison de Performance

In [ ]:
import time

def benchmark_matrix_operations():
    """Comparer les performances avec différentes tailles de matrices"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 Comparaison de performance :")
    print("Taille\t\tTemps CPU (ms)\tCoreML Attendu (ms)")
    print("-" * 56)
    
    for size in sizes:
        # Mesurer le temps d'exécution CPU
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # Temps CoreML attendu (hypothétique)
        # Dans l'implémentation réelle, utiliser les mesures réelles du backend CoreML
        expected_coreml_time = cpu_time * 0.6  # Hypothèse : CoreML est 40% plus rapide
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 Note : Les temps CoreML sont hypothétiques. Les valeurs réelles dépendent de l'implémentation spécifique.")

## Simulation de Sélection de Périphérique

In [ ]:
def simulate_device_selection():
    """Simuler la sélection intelligente de périphérique"""
    
    operations = [
        ("Multiplication de matrice petite", (16, 16), "CPU"),
        ("Multiplication de matrice moyenne", (128, 128), "GPU Metal"),
        ("Multiplication de matrice grande", (512, 512), "CoreML" if coreml_available else "GPU Metal"),
        ("Fonction d'activation", (32, 64, 128, 128), "GPU Metal"),
        ("Petite convolution", (1, 3, 32, 32), "CPU"),
        ("Grande convolution", (16, 64, 224, 224), "CoreML" if coreml_available else "GPU Metal"),
        ("Opérations de nombres complexes", (128, 128), "GPU Metal"),  # CoreML non supporté
        ("Distribution statistique", (1000,), "CPU"),  # CoreML non supporté
    ]
    
    print("🎯 Simulation de sélection intelligente de périphérique :")
    print("Opération\t\t\tForme du Tenseur\t\tPériphérique Sélectionné")
    print("-" * 78)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<31}\t{shape_str:<15}\t{device}")
    
    print("\n📝 Logique de sélection :")
    print("  • Petites opérations : CPU (éviter la surcharge)")
    print("  • Opérations moyennes : GPU Metal (équilibré)")
    print("  • Grandes opérations : CoreML (optimisé)")
    print("  • Opérations non supportées : basculement GPU/CPU")

simulate_device_selection()

## Exemple Pratique : Couche Simple de Réseau Neuronal

In [ ]:
def simulate_neural_network_layer():
    """Simuler une couche de réseau neuronal"""
    
    print("🧠 Simulation de couche de réseau neuronal :")
    
    # Taille du lot et configuration de la couche
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10 classes
    
    print(f"📊 Taille du lot : {batch_size}")
    print(f"🔢 Caractéristiques d'entrée : {input_features}")
    print(f"🧮 Caractéristiques cachées : {hidden_features}")
    print(f"🎯 Caractéristiques de sortie : {output_features}")
    
    # Simulation de passe avant
    steps = [
        ("Entrée → Cachée", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("Activation ReLU", f"({batch_size}, {hidden_features})", "Metal"),
        ("Cachée → Sortie", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Simulation de passe avant :")
    total_time = 0
    
    for step, shape, device in steps:
        # Temps d'exécution virtuel (ms)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ Temps total de la passe avant : {total_time:.2f}ms")
    print(f"🚀 Débit estimé : {1000/total_time:.0f} lots/seconde")

simulate_neural_network_layer()

## Résumé et Prochaines Étapes

In [ ]:
print("📋 Résumé de l'Intégration CoreML RusTorch :")
print()
print("✅ Éléments terminés :")
print("  • Configuration de l'environnement Jupyter")
print("  • Création du noyau Rust et des liaisons Python")
print("  • Vérification de la disponibilité CoreML")
print("  • Gestion des périphériques et configuration")
print("  • Statistiques et profilage du backend")
print("  • Sélection intelligente de périphérique")
print()
print("🚧 Développement futur :")
print("  • Implémentation réelle des opérations CoreML")
print("  • Évaluation comparative des performances")
print("  • Plus de fonctions d'activation et de types de couches")
print("  • Améliorations de la gestion des erreurs")
print("  • Optimisation de la mémoire")
print()
print("🎯 Prochaines étapes recommandées :")
print("  1. Charger et tester de vrais modèles CoreML")
print("  2. Comparer les performances Metal et CoreML")
print("  3. Tester avec de vrais flux de travail d'apprentissage profond")
print("  4. Évaluer en environnement de production")

if coreml_available:
    print("\n🎉 Félicitations ! CoreML est disponible et toutes les fonctionnalités peuvent être testées.")
else:
    print("\n⚠️ CoreML n'est pas disponible, mais les fonctionnalités de base fonctionnent.")
    print("   Nous recommandons de construire avec les fonctionnalités CoreML activées sur macOS.")